<a href="https://colab.research.google.com/github/twhool02/ptm-quantization/blob/main/Pre_trained_model_details.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Show details for base model

This notebook downloads and prints the details of the pre-trained models used in this dissertation


## Setup

### Log into HuggingFace Hub

This code assumes that the user has a hugging face token setup as a notebook secret called HF_TOKEN

In [ ]:
# Required when interacting with HuggingFace Hub
!pip install -q --upgrade huggingface_hub

import huggingface_hub

print(f"Hugging Face Version is: {huggingface_hub.__version__}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 8.0 MB/s eta 0:00:00
Hugging Face Version is: 0.22.2


In [ ]:
from google.colab import userdata

# using the HF_TOKEN secret, this has write permissions to Hugging Face
hftoken = userdata.get('HF_TOKEN')

In [ ]:
from huggingface_hub import login

# Log into hugging face using the HF_TOKEN secrect
login(hftoken, add_to_git_credential=True)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### Install Required Libraries

In [ ]:
# install transformers
!pip install -q -U transformers

# install the stable version of AutoAWQ and it kernelts
!pip install autoawq -q

# accelerate enables the same PyTorch code to be run across any distributed configuration
!pip install -q -U accelerate

# 'bitsandbytes' includes quantization primitives for 8-bit & 4-bit operations
!pip install -q -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 709.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.4/33.4 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 107.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

### Check library versions

In [ ]:
#print the version of transformers
import transformers
print(f"version of transformers: {transformers.__version__}")

#print the version of pytorch
import torch
print(f"version of pytorch: {torch.__version__}")

version of transformers: 4.38.2
version of pytorch: 2.2.1+cu121


### Import Libraries

In [ ]:
# for interacting with the operating system.
import os

# torch is the main package of PyTorch.
import torch

from transformers import (
    AutoModelForCausalLM, # Generic model class with a causal language modeling head
    AutoTokenizer, # Automatically selects correct tokenizer for a model.
    pipeline, # Creates a pipeline that applies a model to some input data.
    logging, # Logs events during training and evaluation.
    BitsAndBytesConfig # transformers class to quantize models
)

# Import required AWQ libraries
from awq import AutoAWQForCausalLM

# provides access to garbage collection
import gc

# relases memory from the GPU
from accelerate.utils import release_memory

# to quantize model
import bitsandbytes

# will be used to measure time
import time


### Define the processor to use

Ensure the model will use a GPU if available

In [ ]:
# Load the model directly onto GPU (if available)
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
# View GPU info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Apr  1 13:39:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              46W / 400W |      5MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

### Define a flush function

This function is to free all allocated memory

In [ ]:
def flush():
  gc.collect()
  torch.cuda.empty_cache()
  torch.cuda.reset_peak_memory_stats()

### Create function to measure memory used

In [ ]:
def bytes_to_gigabytes(bytes):
  return bytes / 1024 / 1024 / 1024

## Load Llama-2-7b-chat-hf model


In-depth details of the model are available at: [meta-llama/Llama-2-7b-chat-hf](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf)

### Load Base Model

In [ ]:
# Define the model to use
model_name = 'meta-llama/Llama-2-7b-chat-hf'

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name, # specifies which pre-trained model to load
    trust_remote_code=True, # allows the execution of remote code.
    device_map=device # load device on to the GPU
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

### View the model details

In [ ]:
print(f"Model size: {model.get_memory_footprint() / 1e9:.1f} GB")
print(f"Model params: {model.num_parameters():,}")
print(f"Model Config: \n{model.config}")
print(f"View model structure: \n{model}")

Model size: 27.1 GB
Model params: 6,738,415,616
Model Config: 
LlamaConfig {
  "_name_or_path": "meta-llama/Llama-2-7b-chat-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.38.2",
  "use_cache": true,
  "vocab_size": 32000
}

View model structure: 
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Lin

In [ ]:
# measure memory used
# torch.cuda.max_memory_allocated - returns the maximum GPU memory occupied by tensors in bytes
# for a given device.
orig_model_mem = bytes_to_gigabytes(torch.cuda.max_memory_allocated(device))
print(f"VRAM usage of {model_name}: {orig_model_mem} GB")

VRAM usage of meta-llama/Llama-2-7b-chat-hf: 25.243194580078125 GB


In [ ]:
# Clear memory usage on the GPU
del model
flush()
release_memory(device)

[None]

## Load Mistral-7B-Instruct

In-depth details of the model are available at:: [Model Card for Mistral-7B-Instruct-v0.2](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2)

### Load Base Model

In [ ]:
# Define the model to use
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name, # specifies which pre-trained model to load
    trust_remote_code=True, # allows the execution of remote code.
    device_map=device # load device on to the GPU
)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

### View model details

In [ ]:
print(f"Model size: {model.get_memory_footprint() / 1e9:.1f} GB")
print(f"Model params: {model.num_parameters():,}")
print(f"Model Config: \n{model.config}")
print(f"View model structure: \n{model}")

Model size: 30.0 GB
Model params: 7,241,732,096
Model Config: 
MistralConfig {
  "_name_or_path": "mistralai/Mistral-7B-Instruct-v0.2",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.38.2",
  "use_cache": true,
  "vocab_size": 32000
}

View model structure: 
MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_

In [ ]:
# measure memory used
# torch.cuda.max_memory_allocated - returns the maximum GPU memory occupied by tensors in bytes
# for a given device.
orig_model_mem = bytes_to_gigabytes(torch.cuda.max_memory_allocated(device))
print(f"VRAM usage of {model_name}: {orig_model_mem} GB")

VRAM usage of mistralai/Mistral-7B-Instruct-v0.2: 27.977569580078125 GB


In [ ]:
# Clear memory usage on the GPU
del model
flush()
release_memory(device)

[None]

## Load Falcon-7b-instruct

In-depth details of the model are available at: [Model Card for Falcon-7B-Instruct](https://huggingface.co/tiiuae/falcon-7b-instruct)

In [ ]:
# Define the model to use
model_name = "tiiuae/falcon-7b-instruct"

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name, # specifies which pre-trained model to load
    trust_remote_code=True, # allows the execution of remote code.
    device_map=device # load device on to the GPU
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### View model details

In [ ]:
print(f"Model size: {model.get_memory_footprint() / 1e9:.1f} GB")
print(f"Model params: {model.num_parameters():,}")
print(f"Model Config: \n{model.config}")
print(f"View model structure: \n{model}")

Model size: 27.7 GB
Model params: 6,921,720,704
Model Config: 
FalconConfig {
  "_name_or_path": "tiiuae/falcon-7b-instruct",
  "alibi": false,
  "apply_residual_connection_post_layernorm": false,
  "architectures": [
    "FalconForCausalLM"
  ],
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "tiiuae/falcon-7b-instruct--configuration_falcon.FalconConfig",
    "AutoModel": "tiiuae/falcon-7b-instruct--modeling_falcon.FalconModel",
    "AutoModelForCausalLM": "tiiuae/falcon-7b-instruct--modeling_falcon.FalconForCausalLM",
    "AutoModelForQuestionAnswering": "tiiuae/falcon-7b-instruct--modeling_falcon.FalconForQuestionAnswering",
    "AutoModelForSequenceClassification": "tiiuae/falcon-7b-instruct--modeling_falcon.FalconForSequenceClassification",
    "AutoModelForTokenClassification": "tiiuae/falcon-7b-instruct--modeling_falcon.FalconForTokenClassification"
  },
  "bias": false,
  "bos_token_id": 11,
  "eos_token_id": 11,
  "hidden_dropout": 0.0,
  "hidden_size": 4544,
  "

In [ ]:
# measure memory used
# torch.cuda.max_memory_allocated - returns the maximum GPU memory occupied by tensors in bytes
# for a given device.
orig_model_mem = bytes_to_gigabytes(torch.cuda.max_memory_allocated(device))
print(f"VRAM usage of {model_name}: {orig_model_mem} GB")

VRAM usage of tiiuae/falcon-7b-instruct: 26.94644546508789 GB


In [ ]:
# Clear memory usage on the GPU
del model
flush()
release_memory(device)

[None]